In [1]:
import os
opj = os.path.join

import ccxt 
import matplotlib.pyplot as plt 
import pandas as pd 
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor 
from sklearn.metrics import f1_score, mean_squared_error, mean_absolute_error
import torch

In [2]:
df_dir = "data/8hours"
df_files = [
    "crypto-gt.csv",
    "deberta.csv",
    "roberta.csv",
    "bert.csv"
]

experiment_name = "sentiment_all_8hours"
saveas = opj("results", f"{experiment_name}.csv")
checkpoint_save_dir = opj("ckpts", experiment_name)
exp_times = 10
results = {}

In [3]:
def experiment(df_dir, df_file, N):
    exp_name = df_file.split(".")[0]
    # load data
    data_dir = opj(df_dir, df_file)
    chart_df = pd.read_csv(data_dir)
    i1 = 0
    i2 = len(chart_df)-1
    st = "2018-02-14 14:00:00"
    en = "2022-04-15 23:00:00"
    filt = []
    for index, row in chart_df.iterrows():
        if chart_df.iloc[i1]['datetime'] < row.datetime and row.datetime < st:
            i1 = index
        if en < row.datetime and row.datetime < chart_df.iloc[i2]['datetime']:
            i2 = index
    chart_df = chart_df[i1+1:i2]
    chart_df.index = range(len(chart_df))
    
    # Train High Model
    ## train/validation/test split 
    train_size = int(chart_df.shape[0] * 0.8) 
    train_df = chart_df.iloc[:train_size,:] 

    val_size = int(chart_df.shape[0] * 0.1) 
    val_df = chart_df.iloc[train_size:train_size+val_size,:]  

    test_df = chart_df.iloc[train_size+val_size:, :] 
    
    categorical_columns = ["months", "days", "hours"]
    features = train_df.columns

    cat_idxs = [0, 1, 2] 
    cat_dims = [13, 32, 25] 

    tabnet_params = {"cat_idxs":cat_idxs, 
                     "cat_dims":cat_dims, 
                     "cat_emb_dim":1, 
                     "optimizer_fn":torch.optim.Adam,
                     "seed": N
                    } 
    
    input_columns = [] 
    for col in train_df.columns:
        if col != 'low_delta' and col != 'years' and col != 'datetime': 
            input_columns.append(col) 

    X_train = train_df[input_columns].values 
    Y_train = train_df['high_delta'].values 
    Y_train = Y_train.reshape((-1,1))

    X_val = val_df[input_columns].values
    Y_val = val_df['high_delta'].values 
    Y_val = Y_val.reshape((-1,1))

    X_test = test_df[input_columns].values 
    Y_test = test_df['high_delta'].values  
    Y_test = Y_test.reshape((-1,1))
    
    reg_high = TabNetRegressor(**tabnet_params) 

    reg_high.fit(X_train, Y_train, 
                 eval_set=[(X_val, Y_val)], 
                 max_epochs=100, 
                 patience=100)  
    
    Y_pred = reg_high.predict(X_test).flatten() 
    
    exp = f"{exp_name}-high"
    if exp not in results:
        results[exp] = []
        
    results[exp].append(
        {
            "MSE": mean_squared_error(Y_test, Y_pred),
            "MAE": mean_absolute_error(Y_test, Y_pred)
        }
    )
    
    saveas = opj(checkpoint_save_dir, f"{exp}-{N}")
    reg_high.save_model(saveas)
    
    # Train Low Model
    input_columns = [] 
    for col in train_df.columns:
        if col != 'high_delta' and col != 'years' and col != 'datetime': 
            input_columns.append(col) 

    X_train = train_df[input_columns].values 
    Y_train = train_df['low_delta'].values 
    Y_train = Y_train.reshape((-1,1))

    X_val = val_df[input_columns].values
    Y_val = val_df['low_delta'].values 
    Y_val = Y_val.reshape((-1,1))

    X_test = test_df[input_columns].values 
    Y_test = test_df['low_delta'].values  
    Y_test = Y_test.reshape((-1,1))
    
    reg_low = TabNetRegressor(**tabnet_params) 

    reg_low.fit(X_train, Y_train, 
                eval_set=[(X_val, Y_val)], 
                max_epochs=200, 
                patience=200)  
    
    Y_pred = reg_low.predict(X_test).flatten() 
    exp = f"{exp_name}-low"
    if exp not in results:
        results[exp] = []
        
    results[exp].append(
        {
            "MSE": mean_squared_error(Y_test, Y_pred),
            "MAE": mean_absolute_error(Y_test, Y_pred)
        }
    )
    
    saveas = opj(checkpoint_save_dir, f"{exp}-{N}")
    reg_high.save_model(saveas)

In [ ]:
for df_file in df_files:
    for i in range(exp_times):
        experiment(df_dir, df_file, i)

Device used : cuda
epoch 0  | loss: 0.2573  | val_0_mse: 0.00587 |  0:00:00s
epoch 1  | loss: 0.03208 | val_0_mse: 0.00136 |  0:00:01s
epoch 2  | loss: 0.00714 | val_0_mse: 0.00081 |  0:00:02s
epoch 3  | loss: 0.00271 | val_0_mse: 0.00015 |  0:00:03s
epoch 4  | loss: 0.00183 | val_0_mse: 0.00014 |  0:00:04s
epoch 5  | loss: 0.00144 | val_0_mse: 0.00014 |  0:00:04s
epoch 6  | loss: 0.00137 | val_0_mse: 0.00018 |  0:00:05s
epoch 7  | loss: 0.00127 | val_0_mse: 0.00015 |  0:00:06s
epoch 8  | loss: 0.00127 | val_0_mse: 0.00015 |  0:00:07s
epoch 9  | loss: 0.00116 | val_0_mse: 0.00015 |  0:00:07s
epoch 10 | loss: 0.00113 | val_0_mse: 0.00015 |  0:00:08s
epoch 11 | loss: 0.00111 | val_0_mse: 0.00015 |  0:00:09s
epoch 12 | loss: 0.00109 | val_0_mse: 0.00015 |  0:00:10s
epoch 13 | loss: 0.00105 | val_0_mse: 0.00014 |  0:00:10s
epoch 14 | loss: 0.00104 | val_0_mse: 0.00013 |  0:00:11s
epoch 15 | loss: 0.00103 | val_0_mse: 0.00015 |  0:00:12s
epoch 16 | loss: 0.00102 | val_0_mse: 0.00015 |  0:00

epoch 37 | loss: 0.00074 | val_0_mse: 0.00017 |  0:00:27s
epoch 38 | loss: 0.00073 | val_0_mse: 0.00016 |  0:00:28s
epoch 39 | loss: 0.00073 | val_0_mse: 0.00017 |  0:00:29s
epoch 40 | loss: 0.00072 | val_0_mse: 0.00016 |  0:00:30s
epoch 41 | loss: 0.0007  | val_0_mse: 0.00016 |  0:00:30s
epoch 42 | loss: 0.0007  | val_0_mse: 0.00016 |  0:00:31s
epoch 43 | loss: 0.00069 | val_0_mse: 0.00016 |  0:00:32s
epoch 44 | loss: 0.00068 | val_0_mse: 0.00015 |  0:00:32s
epoch 45 | loss: 0.00067 | val_0_mse: 0.00016 |  0:00:33s
epoch 46 | loss: 0.00067 | val_0_mse: 0.00016 |  0:00:34s
epoch 47 | loss: 0.00065 | val_0_mse: 0.00015 |  0:00:35s
epoch 48 | loss: 0.00066 | val_0_mse: 0.00015 |  0:00:35s
epoch 49 | loss: 0.00065 | val_0_mse: 0.00015 |  0:00:36s
epoch 50 | loss: 0.00064 | val_0_mse: 0.00015 |  0:00:37s
epoch 51 | loss: 0.00064 | val_0_mse: 0.00015 |  0:00:38s
epoch 52 | loss: 0.00063 | val_0_mse: 0.00015 |  0:00:39s
epoch 53 | loss: 0.00063 | val_0_mse: 0.00015 |  0:00:39s
epoch 54 | los

epoch 179| loss: 0.00036 | val_0_mse: 0.00013 |  0:02:15s
epoch 180| loss: 0.00036 | val_0_mse: 0.00013 |  0:02:16s
epoch 181| loss: 0.00036 | val_0_mse: 0.00014 |  0:02:17s
epoch 182| loss: 0.00037 | val_0_mse: 0.00015 |  0:02:17s
epoch 183| loss: 0.00038 | val_0_mse: 0.00013 |  0:02:18s
epoch 184| loss: 0.00037 | val_0_mse: 0.00013 |  0:02:19s
epoch 185| loss: 0.00036 | val_0_mse: 0.00013 |  0:02:20s
epoch 186| loss: 0.00036 | val_0_mse: 0.00013 |  0:02:20s
epoch 187| loss: 0.00036 | val_0_mse: 0.00013 |  0:02:21s
epoch 188| loss: 0.00035 | val_0_mse: 0.00013 |  0:02:22s
epoch 189| loss: 0.00035 | val_0_mse: 0.00013 |  0:02:23s
epoch 190| loss: 0.00036 | val_0_mse: 0.00013 |  0:02:24s
epoch 191| loss: 0.00035 | val_0_mse: 0.00014 |  0:02:24s
epoch 192| loss: 0.00035 | val_0_mse: 0.00013 |  0:02:25s
epoch 193| loss: 0.00035 | val_0_mse: 0.00013 |  0:02:26s
epoch 194| loss: 0.00035 | val_0_mse: 0.00014 |  0:02:26s
epoch 195| loss: 0.00035 | val_0_mse: 0.00013 |  0:02:27s
epoch 196| los

epoch 12 | loss: 0.00137 | val_0_mse: 0.00018 |  0:00:09s
epoch 13 | loss: 0.0013  | val_0_mse: 0.00019 |  0:00:10s
epoch 14 | loss: 0.00127 | val_0_mse: 0.00017 |  0:00:11s
epoch 15 | loss: 0.00122 | val_0_mse: 0.00017 |  0:00:11s
epoch 16 | loss: 0.0012  | val_0_mse: 0.00017 |  0:00:12s
epoch 17 | loss: 0.00116 | val_0_mse: 0.00017 |  0:00:13s
epoch 18 | loss: 0.00113 | val_0_mse: 0.00016 |  0:00:14s
epoch 19 | loss: 0.00112 | val_0_mse: 0.00016 |  0:00:14s
epoch 20 | loss: 0.0011  | val_0_mse: 0.00016 |  0:00:15s
epoch 21 | loss: 0.0011  | val_0_mse: 0.00017 |  0:00:16s
epoch 22 | loss: 0.00109 | val_0_mse: 0.00016 |  0:00:17s
epoch 23 | loss: 0.00108 | val_0_mse: 0.00017 |  0:00:17s
epoch 24 | loss: 0.00105 | val_0_mse: 0.00016 |  0:00:18s
epoch 25 | loss: 0.00106 | val_0_mse: 0.00016 |  0:00:19s
epoch 26 | loss: 0.00102 | val_0_mse: 0.00018 |  0:00:20s
epoch 27 | loss: 0.00102 | val_0_mse: 0.00016 |  0:00:20s
epoch 28 | loss: 0.00101 | val_0_mse: 0.00016 |  0:00:21s
epoch 29 | los

epoch 154| loss: 0.00041 | val_0_mse: 0.00013 |  0:01:56s
epoch 155| loss: 0.00041 | val_0_mse: 0.00013 |  0:01:57s
epoch 156| loss: 0.00041 | val_0_mse: 0.00013 |  0:01:58s
epoch 157| loss: 0.0004  | val_0_mse: 0.00013 |  0:01:59s
epoch 158| loss: 0.0004  | val_0_mse: 0.00012 |  0:01:59s
epoch 159| loss: 0.0004  | val_0_mse: 0.00013 |  0:02:00s
epoch 160| loss: 0.0004  | val_0_mse: 0.00013 |  0:02:01s
epoch 161| loss: 0.0004  | val_0_mse: 0.00013 |  0:02:02s
epoch 162| loss: 0.0004  | val_0_mse: 0.00013 |  0:02:02s
epoch 163| loss: 0.0004  | val_0_mse: 0.00013 |  0:02:03s
epoch 164| loss: 0.0004  | val_0_mse: 0.00013 |  0:02:04s
epoch 165| loss: 0.00039 | val_0_mse: 0.00013 |  0:02:05s
epoch 166| loss: 0.00039 | val_0_mse: 0.00013 |  0:02:05s
epoch 167| loss: 0.0004  | val_0_mse: 0.00013 |  0:02:06s
epoch 168| loss: 0.0004  | val_0_mse: 0.00013 |  0:02:07s
epoch 169| loss: 0.00039 | val_0_mse: 0.00013 |  0:02:08s
epoch 170| loss: 0.0004  | val_0_mse: 0.00014 |  0:02:08s
epoch 171| los

...

epoch 158| loss: 0.00047 | val_0_mse: 0.00013 |  0:02:10s
epoch 159| loss: 0.00046 | val_0_mse: 0.00012 |  0:02:11s
epoch 160| loss: 0.00045 | val_0_mse: 0.00012 |  0:02:12s
epoch 161| loss: 0.00046 | val_0_mse: 0.00012 |  0:02:13s
epoch 162| loss: 0.00045 | val_0_mse: 0.00012 |  0:02:13s
epoch 163| loss: 0.00045 | val_0_mse: 0.00012 |  0:02:14s
epoch 164| loss: 0.00044 | val_0_mse: 0.00012 |  0:02:15s
epoch 165| loss: 0.00045 | val_0_mse: 0.00012 |  0:02:16s
epoch 166| loss: 0.00045 | val_0_mse: 0.00014 |  0:02:16s
epoch 167| loss: 0.00045 | val_0_mse: 0.00012 |  0:02:17s
epoch 168| loss: 0.00045 | val_0_mse: 0.00012 |  0:02:18s
epoch 169| loss: 0.00044 | val_0_mse: 0.00012 |  0:02:19s
epoch 170| loss: 0.00044 | val_0_mse: 0.00012 |  0:02:20s
epoch 171| loss: 0.00043 | val_0_mse: 0.00012 |  0:02:20s
epoch 172| loss: 0.00043 | val_0_mse: 0.00012 |  0:02:21s
epoch 173| loss: 0.00043 | val_0_mse: 0.00012 |  0:02:22s
epoch 174| loss: 0.00043 | val_0_mse: 0.00012 |  0:02:23s
epoch 175| los

epoch 95 | loss: 0.00051 | val_0_mse: 0.00013 |  0:01:21s
epoch 96 | loss: 0.0005  | val_0_mse: 0.00013 |  0:01:22s
epoch 97 | loss: 0.00049 | val_0_mse: 0.00013 |  0:01:23s
epoch 98 | loss: 0.00049 | val_0_mse: 0.00013 |  0:01:24s
epoch 99 | loss: 0.00049 | val_0_mse: 0.00013 |  0:01:24s
Stop training because you reached max_epochs = 100 with best_epoch = 74 and best_val_0_mse = 0.00013
Best weights from best epoch are automatically used!
Successfully saved model at ckpts/sentiment_all_8hours/roberta-high-4.zip
Device used : cuda
epoch 0  | loss: 2.43348 | val_0_mse: 0.51292 |  0:00:00s
epoch 1  | loss: 0.25938 | val_0_mse: 0.12547 |  0:00:01s
epoch 2  | loss: 0.08468 | val_0_mse: 0.01062 |  0:00:02s
epoch 3  | loss: 0.0328  | val_0_mse: 0.00344 |  0:00:03s
epoch 4  | loss: 0.01217 | val_0_mse: 0.00135 |  0:00:04s
epoch 5  | loss: 0.00579 | val_0_mse: 0.00069 |  0:00:05s
epoch 6  | loss: 0.00304 | val_0_mse: 0.00045 |  0:00:05s
epoch 7  | loss: 0.0022  | val_0_mse: 0.00033 |  0:00:06s

epoch 132| loss: 0.00036 | val_0_mse: 5e-05   |  0:01:58s
epoch 133| loss: 0.00036 | val_0_mse: 5e-05   |  0:01:59s
epoch 134| loss: 0.00036 | val_0_mse: 5e-05   |  0:02:00s
epoch 135| loss: 0.00036 | val_0_mse: 5e-05   |  0:02:01s
epoch 136| loss: 0.00036 | val_0_mse: 5e-05   |  0:02:02s
epoch 137| loss: 0.00037 | val_0_mse: 7e-05   |  0:02:03s
epoch 138| loss: 0.00036 | val_0_mse: 5e-05   |  0:02:04s
epoch 139| loss: 0.00036 | val_0_mse: 7e-05   |  0:02:05s
epoch 140| loss: 0.00038 | val_0_mse: 5e-05   |  0:02:06s
epoch 141| loss: 0.00035 | val_0_mse: 6e-05   |  0:02:07s
epoch 142| loss: 0.00035 | val_0_mse: 5e-05   |  0:02:08s
epoch 143| loss: 0.00035 | val_0_mse: 6e-05   |  0:02:09s
epoch 144| loss: 0.00035 | val_0_mse: 5e-05   |  0:02:10s
epoch 145| loss: 0.00035 | val_0_mse: 6e-05   |  0:02:11s
epoch 146| loss: 0.00034 | val_0_mse: 5e-05   |  0:02:11s
epoch 147| loss: 0.00034 | val_0_mse: 6e-05   |  0:02:12s
epoch 148| loss: 0.00033 | val_0_mse: 6e-05   |  0:02:13s
epoch 149| los

epoch 70 | loss: 0.00048 | val_0_mse: 0.00014 |  0:01:12s
epoch 71 | loss: 0.00047 | val_0_mse: 0.00014 |  0:01:13s
epoch 72 | loss: 0.00047 | val_0_mse: 0.00014 |  0:01:14s
epoch 73 | loss: 0.00047 | val_0_mse: 0.00015 |  0:01:16s
epoch 74 | loss: 0.00047 | val_0_mse: 0.00014 |  0:01:17s
epoch 75 | loss: 0.00045 | val_0_mse: 0.00015 |  0:01:18s
epoch 76 | loss: 0.00045 | val_0_mse: 0.00015 |  0:01:20s
epoch 77 | loss: 0.00045 | val_0_mse: 0.00014 |  0:01:21s
epoch 78 | loss: 0.00044 | val_0_mse: 0.00014 |  0:01:23s
epoch 79 | loss: 0.00044 | val_0_mse: 0.00014 |  0:01:24s
epoch 80 | loss: 0.00043 | val_0_mse: 0.00014 |  0:01:26s
epoch 81 | loss: 0.00043 | val_0_mse: 0.00014 |  0:01:27s
epoch 82 | loss: 0.00042 | val_0_mse: 0.00014 |  0:01:28s
epoch 83 | loss: 0.00041 | val_0_mse: 0.00014 |  0:01:30s
epoch 84 | loss: 0.00041 | val_0_mse: 0.00014 |  0:01:31s
epoch 85 | loss: 0.00041 | val_0_mse: 0.00014 |  0:01:32s
epoch 86 | loss: 0.0004  | val_0_mse: 0.00014 |  0:01:34s
epoch 87 | los

epoch 107| loss: 0.00053 | val_0_mse: 0.00013 |  0:02:13s
epoch 108| loss: 0.00052 | val_0_mse: 0.00013 |  0:02:14s
epoch 109| loss: 0.00052 | val_0_mse: 0.00014 |  0:02:15s
epoch 110| loss: 0.00052 | val_0_mse: 0.00014 |  0:02:16s
epoch 111| loss: 0.00052 | val_0_mse: 0.00013 |  0:02:18s
epoch 112| loss: 0.00051 | val_0_mse: 0.00014 |  0:02:19s
epoch 113| loss: 0.00051 | val_0_mse: 0.00014 |  0:02:20s
epoch 114| loss: 0.00051 | val_0_mse: 0.00014 |  0:02:21s
epoch 115| loss: 0.0005  | val_0_mse: 0.00014 |  0:02:22s
epoch 116| loss: 0.0005  | val_0_mse: 0.00014 |  0:02:23s
epoch 117| loss: 0.0005  | val_0_mse: 0.00013 |  0:02:25s
epoch 118| loss: 0.0005  | val_0_mse: 0.00014 |  0:02:26s
epoch 119| loss: 0.00051 | val_0_mse: 0.00013 |  0:02:27s
epoch 120| loss: 0.0005  | val_0_mse: 0.00013 |  0:02:28s
epoch 121| loss: 0.00049 | val_0_mse: 0.00013 |  0:02:30s
epoch 122| loss: 0.00048 | val_0_mse: 0.00015 |  0:02:31s
epoch 123| loss: 0.00049 | val_0_mse: 0.00013 |  0:02:32s
epoch 124| los

In [ ]:
if not os.path.exists(checkpoint_save_dir):
    os.makedirs(checkpoint_save_dir, exist_ok=True)
with open(saveas, 'w') as f:
    for key in results.keys():
        f.write("%s, %s\n" % (key, results[key]))